# Scraping and Analysing the Texas Death Row Database

Conducted by Dhruv Rachakonda <br>
University of Utah, Salt Lake City, Utah <br>
November 2024

## Introduction

The purpose of this exploratory data analysis (EDA) is to scrape the web table found on the Texas Department of Criminal Justice website, which publishes a table of up-to-date executed inmates with information regarding their execution. 

The following questions will be looked into

<b>Which counties have the most executions?</b> <br> 
<b>What are the most common crimes of those committed on death row?</b> <br>
<b>What are the most common words/phrases in the last words</b>
<b>What are some common patterns in those who are executed</b>

Let us start off by importing the neccesary libraries